In [9]:
import cv2
import os
import time
from pymodbus.client import ModbusTcpClient

In [40]:
coil_address = 1540 #ModBus Y0
input_address = 0
PLC_IP = "192.168.1.111"  #PLC's IP address
PLC_PORT = 502           # Default Modbus TCP port

In [11]:
def modbus(PLC_IP, PLC_PORT):

    # Create Modbus client
    client = ModbusTcpClient(PLC_IP, port=PLC_PORT)

    # Connect to the PLC
    if client.connect():
        print("Connection successful")
    else:
        print("Failed to connect to PLC")
    return client

In [12]:
def PLC(client, coil_address,value):
    write_result = client.write_coil(address=coil_address, value=value)  # Write 'True' to turn ON
    if write_result.isError():
        print("Write Error:", write_result)

In [36]:
def read_PLC_input(client, input_address):
    read_result = client.read_discrete_inputs(address=input_address, count=1)  # Read a single input
    if read_result.isError():
        print("Read Error:", read_result)
        return None
    return read_result.bits[0]  # Return the first (and only) input value


In [63]:
def read_PLC_output(client, coil_address):
    read_result = client.read_coils(address=coil_address)  # Read a single coil (output)
    if read_result.isError():
        print("Read Error:", read_result)
        return None
    return read_result.bits[0]  # Return the first coil state (True for ON, False for OFF)


In [73]:
def capture():
    cap = cv2.VideoCapture(0)
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return
    
    output_dir = "frames"
    frame_count = 0

    flag = True

    #PLC(client, coil_address,True)

    # time.sleep(0.5)  # Wait for PLC to update
    # result = client.read_coils(1540)  # Read the same coil
    # print("Output status:", result.bits[0])

    while True:
        result = client.read_coils(1540)  # Read the same coil
        print("Output status:", result.bits[0])
        if (result.bits[0] and flag) is True:
            print("TRUE")
            ret, frame = cap.read()
            if not ret:
                print("Error: Frame capture failed.")
                break
             
            # Save the current frame as an image
            frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
            cv2.imwrite(frame_filename, frame)
            print(f"Saved: {frame_filename}") 
            frame_count += 1
            flag = False 
            time.sleep(0.5)


            # Display the frame
            cv2.imshow("Live Stream - Capturing Frames", frame)
            time.sleep(1)
    
        if result.bits[0] is False:
            print("FALSE")
            flag = True
            

        

        #Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    #print(f"Captured {frame_count} frames, saved in '{output_dir}'.")

In [ ]:

def save_frames_as_images():
    # Create a directory to store the images
    output_dir = "frames"
    os.makedirs(output_dir, exist_ok=True)

    # Capture video from a camera (or a video file)
    cap = cv2.VideoCapture(2)  # Use 0 for the default camera
    cap.set(cv2.CAP_PROP_FPS, 240)

    # Check if the camera is opened successfully
    if not cap.isOpened():
        print("Error: Could not open the camera.")
        return

    frame_count = 0
    print("Press 'q' to stop capturing frames.")

    duration = 30
    start_time = time.time()

    while True :
        ret, frame = cap.read()
        if not ret:
            print("Error: Frame capture failed.")
            break

        # Save the current frame as an image
        frame_filename = os.path.join(output_dir, f"frame_{frame_count:04d}.jpg")
        cv2.imwrite(frame_filename, frame)
        print(f"Saved: {frame_filename}")
        frame_count += 1
        
        elapsed_time = time.time() - start_time
        if elapsed_time >= duration:
            break

        # Display the frame
        cv2.imshow("Live Stream - Capturing Frames", frame)

        #Break on 'q' key press
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()
    print(f"Captured {frame_count} frames, saved in '{output_dir}'.")

if __name__ == "__main__":
    save_frames_as_images()


In [76]:
if __name__ == "__main__":

    client = modbus(PLC_IP, PLC_PORT)
    capture()
    # import time

    # client.write_coil(1540, True)  # Turn on coil
    # time.sleep(0.5)  # Wait for PLC to update
    # result = client.read_coils(1540)  # Read the same coil
    # print("Output status:", result.bits[0])
    # time.sleep(0.5) 
    # client.write_coil(1540, False)
    # time.sleep(0.5)
    # result = client.read_coils(1540)  # Read the same coil
    # print("Output status:", result.bits[0])

    

Connection successful
Output status: True
TRUE
Saved: frames\frame_0000.jpg
